# Unified Spectral Dynamics Comparison (v2)

Compares three methods for estimating trial-specific spectral dynamics:
1. **Raw Multitaper** (derotated to baseband)
2. **CT-SSMT (LFP-only)**
3. **CT-SSMT (Joint LFP + Spikes)**

## Key Features
- **NEW: Option to use averaged vs final latent estimates**
- **NEW: Posterior variance bands for joint method**
- Proper trial-averaged vs trial-averaged comparison
- Proper trial-specific vs trial-specific comparison
- Amplitude scaling to correct for absolute amplitude mismatches
- Publication-ready figure styling

## Data Format Notes
- **Ground truth Z_lat**: Baseband (unrotated) complex envelope
- **Raw Multitaper (MNE)**: Rotated → we derotate to baseband
- **Joint trace['latent']**: Rotated → we derotate to baseband

In [ ]:
import sys, os, pickle
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from scipy.interpolate import interp1d
from scipy.stats import pearsonr
from simulate_trial_data import TrialSimConfig
WORKSPACE_PATH = "/orcd/data/ekmiller/001/bowen/spike-field-joint-inference"
if WORKSPACE_PATH not in sys.path:
    sys.path.append(WORKSPACE_PATH)

In [ ]:
# Figure Styling
def figure_style(font_size=7):
    """Set publication-quality figure style"""
    sns.set(style="ticks", context="paper",
            font="sans-serif",
            rc={"font.size": font_size,
                "figure.titlesize": font_size,
                "axes.titlesize": font_size,
                "axes.labelsize": font_size,
                "axes.linewidth": 0.5,
                "lines.linewidth": 1,
                "lines.markersize": 3,
                "xtick.labelsize": font_size,
                "ytick.labelsize": font_size,
                "savefig.transparent": True,
                "xtick.major.size": 2.5,
                "ytick.major.size": 2.5,
                "xtick.major.width": 0.5,
                "ytick.major.width": 0.5,
                'legend.fontsize': font_size,
                'legend.title_fontsize': font_size,
                'legend.frameon': False,
            })
    matplotlib.rcParams['pdf.fonttype'] = 42
    matplotlib.rcParams['ps.fonttype'] = 42
    return None

# Updated line widths for better visibility
METHOD_CONFIG = {
    "mt": {"label": "Multi-taper", "color": "#2E86AB", "linestyle": "-", "linewidth": 3},
    "lfp": {"label": "CT-SSMT (LFP-only)", "color": "#A23B72", "linestyle": "-", "linewidth": 3},
    "spk": {"label": "CT-SSMT (Joint)", "color": "#F18F01", "linestyle": "-", "linewidth": 3},
    "gt": {"label": "Ground Truth", "color": "#333333", "linestyle": "-", "linewidth": 2.5},
}

figure_style(font_size=8)

In [ ]:
# ============================================================
# CONFIGURATION
# ============================================================

DATA_PATH = "./data/sim_with_trials.pkl"
JOINT_RESULTS_PATH = "./results/joint_wald_test.pkl"
CTSSMT_RESULTS_PATH = "./results/ctssmt_lfp_only.pkl"
OUTPUT_DIR = "./figures/"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Spectral parameters
FS = 1000.0
WINDOW_SEC = 0.4
NW_PRODUCT = 1
FREQS_DENSE = np.arange(1, 61, 2, dtype=float)

# Visualization parameters
EXAMPLE_TRIALS = [0, 1, 57]
TIME_RANGE = (1.0, 8.0)

# ============================================================
# NEW: CHOOSE BETWEEN AVERAGED OR FINAL LATENT
# ============================================================
# Set to True to use averaged latent across refreshes
# Set to False to use final latent from last refresh
USE_AVERAGED_LATENT = True

print(f"USE_AVERAGED_LATENT = {USE_AVERAGED_LATENT}")

## Helper Functions

In [ ]:
def compute_optimal_scale(estimate, target):
    """Compute optimal scaling factor to match estimate to target."""
    est_flat = estimate.flatten()
    tgt_flat = target.flatten()
    if np.sum(est_flat**2) < 1e-10:
        return 1.0
    return np.dot(tgt_flat, est_flat) / np.dot(est_flat, est_flat)


def compute_metrics(estimate, target, scale=True):
    """Compute MSE and correlation between estimate and target."""
    est_flat = estimate.flatten()
    tgt_flat = target.flatten()
    if scale:
        scale_factor = compute_optimal_scale(estimate, target)
        est_scaled = est_flat * scale_factor
    else:
        scale_factor = 1.0
        est_scaled = est_flat
    mse = np.mean((est_scaled - tgt_flat)**2)
    corr = np.corrcoef(est_flat, tgt_flat)[0, 1]
    return mse, corr, scale_factor


def extract_complex_from_separated(lat_reim, J):
    """Convert SEPARATED format [Re_0..Re_{J-1}, Im_0..Im_{J-1}] to complex."""
    Z_re = lat_reim[:, :, :J]
    Z_im = lat_reim[:, :, J:]
    Z_complex = Z_re + 1j * Z_im
    return np.transpose(Z_complex, (0, 2, 1))


def extract_complex_from_interleaved(fine_state, J, M=1):
    """Convert INTERLEAVED format [Re_0, Im_0, Re_1, Im_1, ...] to complex."""
    fine_state = np.asarray(fine_state)
    if fine_state.ndim == 2:
        T_fine, D = fine_state.shape
        Z = np.zeros((J, T_fine), dtype=complex)
        for j in range(J):
            for m in range(M):
                base = (j * M + m) * 2
                Z[j, :] += fine_state[:, base] + 1j * fine_state[:, base + 1]
        return Z / M
    elif fine_state.ndim == 3:
        R_dim, T_fine, D = fine_state.shape
        Z = np.zeros((R_dim, J, T_fine), dtype=complex)
        for j in range(J):
            for m in range(M):
                base = (j * M + m) * 2
                Z[:, j, :] += fine_state[:, :, base] + 1j * fine_state[:, :, base + 1]
        return Z / M
    else:
        raise ValueError(f"Unexpected shape: {fine_state.shape}")


def extract_variance_from_interleaved(var_state, J, M=1):
    """Extract variance for each frequency from INTERLEAVED format."""
    var_state = np.asarray(var_state)
    if var_state.ndim == 2:
        T_fine, D = var_state.shape
        V = np.zeros((J, T_fine), dtype=float)
        for j in range(J):
            for m in range(M):
                base = (j * M + m) * 2
                V[j, :] += var_state[:, base] + var_state[:, base + 1]
        return V / (M * M)
    elif var_state.ndim == 3:
        R_dim, T_fine, D = var_state.shape
        V = np.zeros((R_dim, J, T_fine), dtype=float)
        for j in range(J):
            for m in range(M):
                base = (j * M + m) * 2
                V[:, j, :] += var_state[:, :, base] + var_state[:, :, base + 1]
        return V / (M * M)
    else:
        raise ValueError(f"Unexpected shape: {var_state.shape}")


def resample_to_target(Z_source, T_target, axis=-1):
    """Resample complex array along time axis to target length."""
    T_source = Z_source.shape[axis]
    if T_source == T_target:
        return Z_source
    t_source = np.linspace(0, 1, T_source)
    t_target = np.linspace(0, 1, T_target)
    if Z_source.ndim == 2:
        J = Z_source.shape[0]
        Z_resampled = np.zeros((J, T_target), dtype=Z_source.dtype)
        for j in range(J):
            if np.iscomplexobj(Z_source):
                f_re = interp1d(t_source, Z_source[j, :].real, kind='linear', fill_value='extrapolate')
                f_im = interp1d(t_source, Z_source[j, :].imag, kind='linear', fill_value='extrapolate')
                Z_resampled[j, :] = f_re(t_target) + 1j * f_im(t_target)
            else:
                f = interp1d(t_source, Z_source[j, :], kind='linear', fill_value='extrapolate')
                Z_resampled[j, :] = f(t_target)
        return Z_resampled
    elif Z_source.ndim == 3:
        R, J = Z_source.shape[:2]
        Z_resampled = np.zeros((R, J, T_target), dtype=Z_source.dtype)
        for r in range(R):
            for j in range(J):
                if np.iscomplexobj(Z_source):
                    f_re = interp1d(t_source, Z_source[r, j, :].real, kind='linear', fill_value='extrapolate')
                    f_im = interp1d(t_source, Z_source[r, j, :].imag, kind='linear', fill_value='extrapolate')
                    Z_resampled[r, j, :] = f_re(t_target) + 1j * f_im(t_target)
                else:
                    f = interp1d(t_source, Z_source[r, j, :], kind='linear', fill_value='extrapolate')
                    Z_resampled[r, j, :] = f(t_target)
        return Z_resampled
    else:
        raise ValueError(f"Unexpected shape: {Z_source.shape}")

## 1. Load Simulation Data

In [ ]:
from simulate_trial_data import TrialSimConfig
print("="*70)
print("LOADING DATA")
print("="*70)

with open(DATA_PATH, "rb") as f:
    data = pickle.load(f)

LFP = data["LFP"]
spikes = data["spikes"]
Z_lat = data["Z_lat"]
freqs_true = data["freqs_hz"]  # All signal bands (coupled + extra)
masks = data["masks"]
time = data["time"]
delta_spk = data.get("delta_spk", 0.001)

# NEW: Extract coupled vs signal-only bands
freqs_coupled = data.get("freqs_hz_coupled", freqs_true[:4])  # Bands that CAN couple to spikes
freqs_extra = data.get("freqs_hz_extra", np.array([]))  # Signal-only bands (never couple)

# Indices in freqs_true for coupled vs signal-only
n_coupled = len(freqs_coupled)
n_extra = len(freqs_extra)

R, T = LFP.shape
_, S, T_fine = spikes.shape
J_true = len(freqs_true)
B = len(FREQS_DENSE)

print(f"LFP: {LFP.shape}")
print(f"Ground truth Z_lat: {Z_lat.shape} (BASEBAND)")
print(f"All signal frequencies: {freqs_true} Hz")
print(f"  Coupled bands ({n_coupled}): {freqs_coupled} Hz")
print(f"  Signal-only bands ({n_extra}): {freqs_extra} Hz")

idx_true_to_dense = np.array([np.argmin(np.abs(FREQS_DENSE - f)) for f in freqs_true])
print(f"True freq -> dense idx: {dict(zip(freqs_true, idx_true_to_dense))}")

# Indices for coupled bands only (for figures 1, 2, 7)
idx_coupled_to_dense = np.array([np.argmin(np.abs(FREQS_DENSE - f)) for f in freqs_coupled])
idx_coupled_in_true = np.arange(n_coupled)  # First n_coupled bands in freqs_true

Z_gt = np.zeros((R, B, T), dtype=complex)
for j_true, j_dense in enumerate(idx_true_to_dense):
    Z_gt[:, j_dense, :] = Z_lat[:, j_true, :]

print(f"Ground truth expanded: {Z_gt.shape}")

## 2. Compute Raw Multitaper (DEROTATED)

In [ ]:
import mne
from src.utils_multitaper import derotate_tfr_align_start

def compute_multitaper_derotated(lfp, fs, freqs, window_sec, NW_product):
    R, T = lfp.shape
    tfr_raw = mne.time_frequency.tfr_array_multitaper(
        lfp[:, None, :], sfreq=fs, freqs=freqs, n_cycles=freqs * window_sec,
        time_bandwidth=2 * NW_product, output="complex", zero_mean=False,
    ).squeeze()
    tfr_raw_mt = tfr_raw[:, :, None, :]
    M = int(round(window_sec * fs))
    tfr_derotated = derotate_tfr_align_start(tfr_raw_mt, freqs, fs, 1, M)
    tapers, _ = mne.time_frequency.multitaper.dpss_windows(M, NW_product, Kmax=1)
    tfr_derotated = tfr_derotated * (2.0 / tapers.sum(axis=1))
    return tfr_derotated.squeeze(axis=2), tfr_raw

print("Recomputing multitaper with derotation...")
Z_mt, tfr_raw_rotated = compute_multitaper_derotated(LFP, FS, FREQS_DENSE, WINDOW_SEC, NW_PRODUCT)
print(f"Z_mt (derotated): {Z_mt.shape}")

## 3. Load CT-SSMT (LFP-only) Results

In [ ]:
Z_lfp = None
HAS_LFP = False

if os.path.exists(CTSSMT_RESULTS_PATH):
    with open(CTSSMT_RESULTS_PATH, 'rb') as f:
        ctssmt_results = pickle.load(f)
    print(f"Keys: {list(ctssmt_results.keys())}")
    if 'Z_smooth_full' in ctssmt_results:
        Z_lfp = ctssmt_results['Z_smooth_full']
    elif 'Z_smooth' in ctssmt_results:
        Z_lfp = ctssmt_results['Z_smooth']
    if Z_lfp is not None:
        Z_lfp = resample_to_target(Z_lfp, T)
        print(f"CT-SSMT (LFP-only): {Z_lfp.shape}")
        HAS_LFP = True
else:
    print(f"CT-SSMT results not found at {CTSSMT_RESULTS_PATH}")

## 4. Load Joint Inference Results

In [ ]:
Z_spk = None
X_shared = None
D_deviation = None
Z_spk_var = None
HAS_SPK = False

def derotate_tfr(tfr_rotated, freqs, time):
    R, J, T_dim = tfr_rotated.shape
    tfr_baseband = np.zeros_like(tfr_rotated)
    for j, f in enumerate(freqs):
        phase = 2.0 * np.pi * f * time
        tfr_baseband[:, j, :] = tfr_rotated[:, j, :] * np.exp(-1j * phase)[None, :]
    return tfr_baseband

if os.path.exists(JOINT_RESULTS_PATH):
    with open(JOINT_RESULTS_PATH, "rb") as f:
        joint_results = pickle.load(f)
    trace = joint_results['trace']
    freqs_joint = joint_results['freqs_dense']
    J_joint = len(freqs_joint)
    print(f"Trace keys: {list(trace.keys())}")
    
    if 'latent' in trace and len(trace['latent']) > 0:
        lat_reim = np.asarray(trace['latent'][-1])
        print(f"  trace['latent'] shape: {lat_reim.shape}")
        R_lat, T_lat, twoJ = lat_reim.shape
        J_lat = twoJ // 2
        if 'latent_scale_factors' in trace:
            scale_factors = np.asarray(trace['latent_scale_factors'])
            print(f"  Rescaling by scale_factors")
            lat_reim = lat_reim * scale_factors[None, None, :]
        Z_joint_rotated = extract_complex_from_separated(lat_reim, J_lat)
        Z_joint_rotated = resample_to_target(Z_joint_rotated, T)
        Z_spk = derotate_tfr(Z_joint_rotated, freqs_joint, time)
        print(f"  Joint (derotated): {Z_spk.shape}")
        HAS_SPK = True
    
    # Load X_fine and D_fine (averaged or final)
    if USE_AVERAGED_LATENT:
        X_key = 'X_fine_avg' if 'X_fine_avg' in trace else 'X_fine'
        D_key = 'D_fine_avg' if 'D_fine_avg' in trace else 'D_fine'
        print(f"  Using AVERAGED latent: {X_key}, {D_key}")
    else:
        X_key = 'X_fine_final' if 'X_fine_final' in trace else 'X_fine'
        D_key = 'D_fine_final' if 'D_fine_final' in trace else 'D_fine'
        print(f"  Using FINAL latent: {X_key}, {D_key}")
    
    if X_key in trace:
        X_fine_raw = trace[X_key]
        X_fine = np.asarray(X_fine_raw[-1] if isinstance(X_fine_raw, list) else X_fine_raw)
        print(f"  {X_key} shape: {X_fine.shape}")
    else:
        X_fine = None
    
    if D_key in trace:
        D_fine_raw = trace[D_key]
        D_fine = np.asarray(D_fine_raw[-1] if isinstance(D_fine_raw, list) else D_fine_raw)
        print(f"  {D_key} shape: {D_fine.shape}")
    else:
        D_fine = None
    
    if X_fine is not None and D_fine is not None:
        X_shared = extract_complex_from_interleaved(X_fine, J_joint)
        D_deviation = extract_complex_from_interleaved(D_fine, J_joint)
        X_shared = resample_to_target(X_shared, T)
        D_deviation = resample_to_target(D_deviation, T)
        print(f"  X_shared: {X_shared.shape}")
        print(f"  D_deviation: {D_deviation.shape}")
    
    # Load posterior variance
    X_var_key = 'X_var_fine_final' if 'X_var_fine_final' in trace else 'X_var_fine'
    D_var_key = 'D_var_fine_final' if 'D_var_fine_final' in trace else 'D_var_fine'
    X_var_fine, D_var_fine = None, None
    
    if X_var_key in trace:
        X_var_raw = trace[X_var_key]
        X_var_fine = np.asarray(X_var_raw[-1] if isinstance(X_var_raw, list) else X_var_raw)
        print(f"  {X_var_key} shape: {X_var_fine.shape}")
    if D_var_key in trace:
        D_var_raw = trace[D_var_key]
        D_var_fine = np.asarray(D_var_raw[-1] if isinstance(D_var_raw, list) else D_var_raw)
        print(f"  {D_var_key} shape: {D_var_fine.shape}")
    
    if X_var_fine is not None and D_var_fine is not None:
        X_var_freq = extract_variance_from_interleaved(X_var_fine, J_joint)
        D_var_freq = extract_variance_from_interleaved(D_var_fine, J_joint)
        Z_spk_var = X_var_freq[None, :, :] + D_var_freq
        Z_spk_var = resample_to_target(Z_spk_var, T)
        print(f"  Z_spk_var (posterior variance): {Z_spk_var.shape}")
    else:
        print(f"  WARNING: Posterior variance not available")
else:
    print(f"Joint results not found at {JOINT_RESULTS_PATH}")

In [ ]:
# Summary
print("="*70)
print("DATA SUMMARY")
print("="*70)
print(f"Ground truth: {Z_gt.shape}")
print(f"Multitaper:   {Z_mt.shape}")
print(f"LFP-only:     {Z_lfp.shape if HAS_LFP else 'N/A'}")
print(f"Joint:        {Z_spk.shape if HAS_SPK else 'N/A'}")
print(f"X_shared:     {X_shared.shape if X_shared is not None else 'N/A'}")
print(f"D_deviation:  {D_deviation.shape if D_deviation is not None else 'N/A'}")
print(f"Z_spk_var:    {Z_spk_var.shape if Z_spk_var is not None else 'N/A'}")

## Figure 1: Trial-Specific Comparison (COUPLED BANDS ONLY)

In [ ]:
# ============================================================
# PLOT 1: Trial-Specific Comparison (4 sample trials)
# ONLY SHOW COUPLED BANDS (freqs_coupled)
# ============================================================

figure_style(font_size=8)
SAMPLE_TRIALS = [0, 25, 57, 85]

def compute_global_scale(estimate, target, freq_indices):
    est_flat = np.abs(estimate[:, freq_indices, :]).flatten()
    tgt_flat = np.abs(target[:, freq_indices, :]).flatten()
    if np.sum(est_flat**2) < 1e-10:
        return 1.0
    return np.dot(tgt_flat, est_flat) / np.dot(est_flat, est_flat)

scale_mt_global = compute_global_scale(Z_mt, Z_gt, idx_true_to_dense)
scale_lfp_global = compute_global_scale(Z_lfp, Z_gt, idx_true_to_dense) if HAS_LFP else 1.0
scale_spk_global = compute_global_scale(Z_spk, Z_gt, idx_true_to_dense) if HAS_SPK else 1.0

print(f"Global scale factors: MT={scale_mt_global:.4f}, LFP={scale_lfp_global:.4f}, Joint={scale_spk_global:.4f}")

# Only plot COUPLED bands
n_freqs_plot = len(freqs_coupled)
n_trials = len(SAMPLE_TRIALS)
fig, axes = plt.subplots(n_freqs_plot, n_trials, figsize=(3.5*n_trials, 2.5*n_freqs_plot), squeeze=False)

t_mask = (time >= TIME_RANGE[0]) & (time <= TIME_RANGE[1])
t_plot = time[t_mask]

for i, (j_true, freq_hz) in enumerate(zip(idx_coupled_in_true, freqs_coupled)):
    j_dense = idx_true_to_dense[j_true]
    for j, trial_idx in enumerate(SAMPLE_TRIALS):
        ax = axes[i, j]
        amp_gt_trial = np.abs(Z_gt[trial_idx, j_dense, t_mask])
        ax.plot(t_plot, amp_gt_trial, color=METHOD_CONFIG['gt']['color'], 
                lw=METHOD_CONFIG['gt']['linewidth'], label='Ground Truth', zorder=5)
        
        amp_mt_trial = np.abs(Z_mt[trial_idx, j_dense, t_mask]) * scale_mt_global
        ax.plot(t_plot, amp_mt_trial, color=METHOD_CONFIG['mt']['color'], 
                lw=METHOD_CONFIG['mt']['linewidth'], label=METHOD_CONFIG['mt']['label'], alpha=0.6, zorder=1)
        
        if HAS_LFP:
            amp_lfp_trial = np.abs(Z_lfp[trial_idx, j_dense, t_mask]) * scale_lfp_global
            ax.plot(t_plot, amp_lfp_trial, color=METHOD_CONFIG['lfp']['color'], 
                    lw=METHOD_CONFIG['lfp']['linewidth'], label=METHOD_CONFIG['lfp']['label'], alpha=0.7, zorder=2)
        
        if HAS_SPK:
            amp_spk_trial = np.abs(Z_spk[trial_idx, j_dense, t_mask]) * scale_spk_global
            ax.plot(t_plot, amp_spk_trial, color=METHOD_CONFIG['spk']['color'], 
                    lw=METHOD_CONFIG['spk']['linewidth'], label=METHOD_CONFIG['spk']['label'], alpha=0.8, zorder=3)
            if Z_spk_var is not None:
                std_spk_trial = np.sqrt(Z_spk_var[trial_idx, j_dense, t_mask]) * scale_spk_global
                ax.fill_between(t_plot, amp_spk_trial - std_spk_trial, amp_spk_trial + std_spk_trial,
                                color=METHOD_CONFIG['spk']['color'], alpha=0.2, zorder=0)
        
        ax.set_xlim(TIME_RANGE)
        if i == 0: ax.set_title(f'Trial {trial_idx}', fontsize=9, fontweight='bold')
        if j == 0: ax.set_ylabel(f'{freq_hz:.0f} Hz\nAmplitude', fontsize=8)
        if i == 0 and j == n_trials - 1: ax.legend(loc='upper right', fontsize=6)
        if i == n_freqs_plot - 1: ax.set_xlabel('Time (s)', fontsize=8)

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'trial_specific_4trials.pdf'), dpi=150, bbox_inches='tight')
plt.savefig(os.path.join(OUTPUT_DIR, 'trial_specific_4trials.png'), dpi=150, bbox_inches='tight')
plt.show()

print("\nTrial-specific correlations with ground truth (COUPLED BANDS):")
print(f"{'Trial':<8} {'Freq':<8} {'MT':>10} {'LFP':>10} {'Joint':>10}")
print("-"*50)
for trial_idx in SAMPLE_TRIALS:
    for j_true, freq_hz in zip(idx_coupled_in_true, freqs_coupled):
        j_dense = idx_true_to_dense[j_true]
        gt = np.abs(Z_gt[trial_idx, j_dense, :])
        r_mt = np.corrcoef(gt, np.abs(Z_mt[trial_idx, j_dense, :]))[0, 1]
        r_lfp = np.corrcoef(gt, np.abs(Z_lfp[trial_idx, j_dense, :]))[0, 1] if HAS_LFP else np.nan
        r_spk = np.corrcoef(gt, np.abs(Z_spk[trial_idx, j_dense, :]))[0, 1] if HAS_SPK else np.nan
        print(f"{trial_idx:<8} {freq_hz:.0f} Hz{'':<3} {r_mt:>10.3f} {r_lfp:>10.3f} {r_spk:>10.3f}")

## Figure 2: Deviation Term Comparison (COUPLED BANDS ONLY)

In [ ]:
# ============================================================
# PLOT 2: Deviation Term Comparison
# ONLY SHOW COUPLED BANDS (freqs_coupled)
# ============================================================

figure_style(font_size=8)

Z_gt_mean = np.abs(Z_gt).mean(axis=0)
Z_mt_mean = np.abs(Z_mt).mean(axis=0)
Z_lfp_mean = np.abs(Z_lfp).mean(axis=0) if HAS_LFP else None
Z_spk_mean = np.abs(Z_spk).mean(axis=0) if HAS_SPK else None

Z_gt_dev = np.abs(Z_gt) - Z_gt_mean[None, :, :]
Z_mt_dev = np.abs(Z_mt) - Z_mt_mean[None, :, :]
Z_lfp_dev = np.abs(Z_lfp) - Z_lfp_mean[None, :, :] if HAS_LFP else None
Z_spk_dev = np.abs(Z_spk) - Z_spk_mean[None, :, :] if HAS_SPK else None

D_amplitude = np.abs(D_deviation) if D_deviation is not None else None

def compute_dev_scale(dev_est, dev_gt, freq_indices):
    est = dev_est[:, freq_indices, :].flatten()
    gt = dev_gt[:, freq_indices, :].flatten()
    if np.sum(np.abs(est)**2) < 1e-10:
        return 1.0
    return np.dot(np.abs(gt), np.abs(est)) / np.dot(np.abs(est), np.abs(est))

scale_mt_dev = compute_dev_scale(Z_mt_dev, Z_gt_dev, idx_true_to_dense)
scale_lfp_dev = compute_dev_scale(Z_lfp_dev, Z_gt_dev, idx_true_to_dense) if HAS_LFP else 1.0
scale_spk_dev = compute_dev_scale(Z_spk_dev, Z_gt_dev, idx_true_to_dense) if HAS_SPK else 1.0

print(f"Deviation scale factors: MT={scale_mt_dev:.4f}, LFP={scale_lfp_dev:.4f}, Joint={scale_spk_dev:.4f}")

# Only plot COUPLED bands
n_freqs_plot = len(freqs_coupled)
fig, axes = plt.subplots(n_freqs_plot, n_trials, figsize=(3.5*n_trials, 2.5*n_freqs_plot), squeeze=False)

for i, (j_true, freq_hz) in enumerate(zip(idx_coupled_in_true, freqs_coupled)):
    j_dense = idx_true_to_dense[j_true]
    for j, trial_idx in enumerate(SAMPLE_TRIALS):
        ax = axes[i, j]
        dev_gt_trial = Z_gt_dev[trial_idx, j_dense, t_mask]
        ax.plot(t_plot, dev_gt_trial, color=METHOD_CONFIG['gt']['color'], 
                lw=METHOD_CONFIG['gt']['linewidth'], label='GT Deviation', zorder=5)
        
        dev_mt_trial = Z_mt_dev[trial_idx, j_dense, t_mask] * scale_mt_dev
        ax.plot(t_plot, dev_mt_trial, color=METHOD_CONFIG['mt']['color'], 
                lw=METHOD_CONFIG['mt']['linewidth'], label='MT Deviation', alpha=0.6, zorder=1)
        
        if HAS_LFP and Z_lfp_dev is not None:
            dev_lfp_trial = Z_lfp_dev[trial_idx, j_dense, t_mask] * scale_lfp_dev
            ax.plot(t_plot, dev_lfp_trial, color=METHOD_CONFIG['lfp']['color'], 
                    lw=METHOD_CONFIG['lfp']['linewidth'], label='LFP Deviation', alpha=0.7, zorder=2)
        
        if HAS_SPK and Z_spk_dev is not None:
            dev_spk_trial = Z_spk_dev[trial_idx, j_dense, t_mask] * scale_spk_dev
            ax.plot(t_plot, dev_spk_trial, color=METHOD_CONFIG['spk']['color'], 
                    lw=METHOD_CONFIG['spk']['linewidth'], label='Joint Deviation', alpha=0.8, zorder=3)
            if Z_spk_var is not None:
                std_spk_trial = np.sqrt(Z_spk_var[trial_idx, j_dense, t_mask]) * scale_spk_dev
                ax.fill_between(t_plot, dev_spk_trial - std_spk_trial, dev_spk_trial + std_spk_trial,
                                color=METHOD_CONFIG['spk']['color'], alpha=0.2, zorder=0)
        
        ax.axhline(0, color='gray', linestyle='--', lw=0.8, alpha=0.5)
        ax.set_xlim(TIME_RANGE)
        if i == 0: ax.set_title(f'Trial {trial_idx}', fontsize=9, fontweight='bold')
        if j == 0: ax.set_ylabel(f'{freq_hz:.0f} Hz\nDeviation', fontsize=8)
        if i == 0 and j == n_trials - 1: ax.legend(loc='upper right', fontsize=6)
        if i == n_freqs_plot - 1: ax.set_xlabel('Time (s)', fontsize=8)

plt.suptitle('Trial-Specific Deviation: estimate - trial_mean (Coupled Bands)', fontsize=10, y=1.01)
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'deviation_comparison_4trials.pdf'), dpi=150, bbox_inches='tight')
plt.savefig(os.path.join(OUTPUT_DIR, 'deviation_comparison_4trials.png'), dpi=150, bbox_inches='tight')
plt.show()

print("\nDeviation correlations with ground truth deviation (COUPLED BANDS):")
print(f"{'Trial':<8} {'Freq':<8} {'MT':>10} {'LFP':>10} {'Joint':>10}")
print("-"*50)
for trial_idx in SAMPLE_TRIALS:
    for j_true, freq_hz in zip(idx_coupled_in_true, freqs_coupled):
        j_dense = idx_true_to_dense[j_true]
        gt_dev = Z_gt_dev[trial_idx, j_dense, :]
        r_mt = np.corrcoef(gt_dev, Z_mt_dev[trial_idx, j_dense, :])[0, 1]
        r_lfp = np.corrcoef(gt_dev, Z_lfp_dev[trial_idx, j_dense, :])[0, 1] if HAS_LFP else np.nan
        r_spk = np.corrcoef(gt_dev, Z_spk_dev[trial_idx, j_dense, :])[0, 1] if HAS_SPK else np.nan
        print(f"{trial_idx:<8} {freq_hz:.0f} Hz{'':<3} {r_mt:>10.3f} {r_lfp:>10.3f} {r_spk:>10.3f}")

print("\n" + "="*50)
print("AGGREGATE DEVIATION METRICS (Coupled Bands)")
print("="*50)
dev_gt_all = Z_gt_dev[:, idx_coupled_to_dense, :].flatten()
dev_mt_all = Z_mt_dev[:, idx_coupled_to_dense, :].flatten()
print(f"Multi-taper deviation correlation:    r = {np.corrcoef(dev_gt_all, dev_mt_all)[0, 1]:.3f}")
if HAS_LFP:
    dev_lfp_all = Z_lfp_dev[:, idx_coupled_to_dense, :].flatten()
    print(f"CT-SSMT (LFP) deviation correlation:  r = {np.corrcoef(dev_gt_all, dev_lfp_all)[0, 1]:.3f}")
if HAS_SPK:
    dev_spk_all = Z_spk_dev[:, idx_coupled_to_dense, :].flatten()
    print(f"CT-SSMT (Joint) deviation correlation: r = {np.corrcoef(dev_gt_all, dev_spk_all)[0, 1]:.3f}")

## Figure 3: Trial-Averaged Comparison (SEPARATE PLOTS PER FREQUENCY)

In [ ]:
# ============================================================
# PLOT 3: Trial-Averaged Comparison - SEPARATE PLOTS
# Save to ./figures/trial_average_dynamics/
# ============================================================

figure_style(font_size=8)

# Create output directory for trial-averaged dynamics
TRIAL_AVG_DIR = os.path.join(OUTPUT_DIR, 'trial_average_dynamics')
os.makedirs(TRIAL_AVG_DIR, exist_ok=True)

# Generate separate plot for each frequency
for j_true, freq_hz in enumerate(freqs_true):
    j_dense = idx_true_to_dense[j_true]
    
    fig, ax = plt.subplots(figsize=(8, 4))
    
    amp_gt = np.abs(Z_gt[:, j_dense, :])[:, t_mask]
    mean_gt = amp_gt.mean(axis=0)
    std_gt = amp_gt.std(axis=0)
    ax.plot(t_plot, mean_gt, color=METHOD_CONFIG['gt']['color'], 
            lw=METHOD_CONFIG['gt']['linewidth'], label='Ground Truth')
    ax.fill_between(t_plot, mean_gt - std_gt, mean_gt + std_gt, 
                    color=METHOD_CONFIG['gt']['color'], alpha=0.15)
    
    amp_mt = np.abs(Z_mt[:, j_dense, :])[:, t_mask]
    mean_mt = amp_mt.mean(axis=0)
    scale_mt = compute_optimal_scale(mean_mt, mean_gt)
    ax.plot(t_plot, mean_mt * scale_mt, color=METHOD_CONFIG['mt']['color'], 
            lw=METHOD_CONFIG['mt']['linewidth'], label=METHOD_CONFIG['mt']['label'], alpha=0.7)
    
    if HAS_LFP:
        amp_lfp = np.abs(Z_lfp[:, j_dense, :])[:, t_mask]
        mean_lfp = amp_lfp.mean(axis=0)
        scale_lfp = compute_optimal_scale(mean_lfp, mean_gt)
        ax.plot(t_plot, mean_lfp * scale_lfp, color=METHOD_CONFIG['lfp']['color'], 
                lw=METHOD_CONFIG['lfp']['linewidth'], label=METHOD_CONFIG['lfp']['label'], alpha=0.8)
    
    if HAS_SPK:
        amp_spk = np.abs(Z_spk[:, j_dense, :])[:, t_mask]
        mean_spk = amp_spk.mean(axis=0)
        scale_spk = compute_optimal_scale(mean_spk, mean_gt)
        ax.plot(t_plot, mean_spk * scale_spk, color=METHOD_CONFIG['spk']['color'], 
                lw=METHOD_CONFIG['spk']['linewidth'], label=METHOD_CONFIG['spk']['label'], alpha=0.9)
    
    ax.set_ylabel('Amplitude', fontsize=10)
    ax.set_xlabel('Time (s)', fontsize=10)
    ax.set_title(f'{freq_hz:.0f} Hz - Trial Averaged', fontsize=12)
    ax.legend(loc='upper right', fontsize=8)
    ax.set_xlim(TIME_RANGE)
    
    # Mark if this is a coupled or signal-only band
    is_coupled = freq_hz in freqs_coupled
    band_type = "coupled" if is_coupled else "signal_only"
    
    sns.despine()
    plt.tight_layout()
    
    # Save with frequency and band type in filename
    plt.savefig(os.path.join(TRIAL_AVG_DIR, f'trial_averaged_{int(freq_hz)}Hz_{band_type}.pdf'), 
                dpi=150, bbox_inches='tight')
    plt.savefig(os.path.join(TRIAL_AVG_DIR, f'trial_averaged_{int(freq_hz)}Hz_{band_type}.png'), 
                dpi=150, bbox_inches='tight')
    plt.show()
    
print(f"\nSaved {len(freqs_true)} separate trial-averaged plots to {TRIAL_AVG_DIR}")

## Summary Metrics

In [ ]:
print("="*70)
print("AGGREGATE AMPLITUDE CORRELATIONS")
print("="*70)
for j_true, freq_hz in enumerate(freqs_true):
    j_dense = idx_true_to_dense[j_true]
    gt_flat = np.abs(Z_gt[:, j_dense, :]).flatten()
    r_mt = np.corrcoef(gt_flat, np.abs(Z_mt[:, j_dense, :]).flatten())[0, 1]
    r_lfp = np.corrcoef(gt_flat, np.abs(Z_lfp[:, j_dense, :]).flatten())[0, 1] if HAS_LFP else np.nan
    r_spk = np.corrcoef(gt_flat, np.abs(Z_spk[:, j_dense, :]).flatten())[0, 1] if HAS_SPK else np.nan
    is_coupled = freq_hz in freqs_coupled
    tag = "(coupled)" if is_coupled else "(signal-only)"
    print(f"{freq_hz:.0f} Hz {tag}: MT={r_mt:.3f}, LFP={r_lfp:.3f}, Joint={r_spk:.3f}")

gt_all = np.abs(Z_gt[:, idx_true_to_dense, :]).flatten()
mt_all = np.abs(Z_mt[:, idx_true_to_dense, :]).flatten()
r_mt_all = np.corrcoef(gt_all, mt_all)[0, 1]
r_lfp_all = np.corrcoef(gt_all, np.abs(Z_lfp[:, idx_true_to_dense, :]).flatten())[0, 1] if HAS_LFP else np.nan
r_spk_all = np.corrcoef(gt_all, np.abs(Z_spk[:, idx_true_to_dense, :]).flatten())[0, 1] if HAS_SPK else np.nan
print("-"*50)
print(f"OVERALL: MT={r_mt_all:.3f}, LFP={r_lfp_all:.3f}, Joint={r_spk_all:.3f}")

## X/D Variance Decomposition

In [ ]:
if X_shared is not None and D_deviation is not None:
    print("="*70)
    print("X/D VARIANCE DECOMPOSITION")
    print("="*70)
    print(f"{'Frequency':<12} {'X var %':<12} {'D var %':<12} {'r(X, GT)':<12} {'r(X+D, GT)':<12}")
    print("-"*70)
    for j_true, freq_hz in enumerate(freqs_true):
        j_dense = idx_true_to_dense[j_true]
        amp_gt = np.abs(Z_gt[:, j_dense, :]).flatten()
        amp_X = np.abs(np.broadcast_to(X_shared[j_dense:j_dense+1, :], (R, T))).flatten()
        Z_XD_here = X_shared[j_dense, :] + D_deviation[:, j_dense, :]
        amp_XD = np.abs(Z_XD_here).flatten()
        var_X = np.var(np.abs(X_shared[j_dense, :]))
        var_D = np.var(np.abs(D_deviation[:, j_dense, :]))
        total_var = var_X + var_D
        pct_X = 100 * var_X / total_var if total_var > 0 else 0
        pct_D = 100 * var_D / total_var if total_var > 0 else 0
        r_X = np.corrcoef(amp_gt, amp_X)[0, 1]
        r_XD = np.corrcoef(amp_gt, amp_XD)[0, 1]
        print(f"{freq_hz:.0f} Hz{'':<6} {pct_X:<12.1f} {pct_D:<12.1f} {r_X:<12.3f} {r_XD:<12.3f}")
    print("-"*70)
    amp_gt_all = np.abs(Z_gt[:, idx_true_to_dense, :]).flatten()
    amp_X_all = np.abs(np.broadcast_to(X_shared[idx_true_to_dense, :][None, :, :], (R, J_true, T))).flatten()
    Z_XD_all = X_shared[idx_true_to_dense, :] + D_deviation[:, idx_true_to_dense, :]
    amp_XD_all = np.abs(Z_XD_all).flatten()
    r_X_agg = np.corrcoef(amp_gt_all, amp_X_all)[0, 1]
    r_XD_agg = np.corrcoef(amp_gt_all, amp_XD_all)[0, 1]
    print(f"{'AGGREGATE':<12} {'-':<12} {'-':<12} {r_X_agg:<12.3f} {r_XD_agg:<12.3f}")
    if r_XD_agg > r_X_agg + 0.01:
        print("\n✓ D contributes meaningful trial-specific information!")
    else:
        print("\n✗ D does not add much beyond X (shared dynamics dominate)")
else:
    print("X/D decomposition not available (no X_fine/D_fine in trace)")

## Figure 4: Spectrogram Heatmaps - Trial-Specific

In [ ]:
# ============================================================
# PLOT 4: Spectrogram Heatmaps - Trial-Specific
# ============================================================

figure_style(font_size=8)
SAMPLE_TRIALS_SPEC = [0, 25, 57]
n_trials_spec = len(SAMPLE_TRIALS_SPEC)

t_mask_spec = (time >= TIME_RANGE[0]) & (time <= TIME_RANGE[1])
t_plot_spec = time[t_mask_spec]
t_start_idx = np.where(t_mask_spec)[0][0]
t_end_idx = np.where(t_mask_spec)[0][-1] + 1

methods = ['gt', 'mt']
method_labels = ['Ground Truth', 'Multi-taper']
method_data = [Z_gt, Z_mt]
if HAS_LFP:
    methods.append('lfp'); method_labels.append('CT-SSMT (LFP)'); method_data.append(Z_lfp)
if HAS_SPK:
    methods.append('spk'); method_labels.append('CT-SSMT (Joint)'); method_data.append(Z_spk)
n_methods = len(methods)

method_vmax, method_vmin = [], []
for data in method_data:
    all_amps = [np.abs(data[trial_idx, :, t_start_idx:t_end_idx]) for trial_idx in SAMPLE_TRIALS_SPEC]
    method_vmax.append(np.percentile(np.concatenate([a.flatten() for a in all_amps]), 99.9))
    method_vmin.append(np.percentile(np.concatenate([a.flatten() for a in all_amps]), 5))

fig, axes = plt.subplots(n_trials_spec, n_methods, figsize=(3.5*n_methods, 2.5*n_trials_spec))
extent = [t_plot_spec[0], t_plot_spec[-1], FREQS_DENSE[0], FREQS_DENSE[-1]]

for i, trial_idx in enumerate(SAMPLE_TRIALS_SPEC):
    for j, (method, label, data, vmax, vmin) in enumerate(zip(methods, method_labels, method_data, method_vmax, method_vmin)):
        ax = axes[i, j]
        amp = np.abs(data[trial_idx, :, t_start_idx:t_end_idx])
        im = ax.imshow(amp, aspect='auto', origin='lower', extent=extent, cmap='viridis', vmin=vmin, vmax=vmax, interpolation='none')
        for freq_true in freqs_true:
            ax.axhline(freq_true, color='white', linestyle='--', lw=0.8, alpha=0.7)
        if i == 0: ax.set_title(label, fontsize=10, fontweight='bold')
        if j == 0: ax.set_ylabel(f'Trial {trial_idx}\nFreq (Hz)', fontsize=9)
        else: ax.set_yticks([])
        if i == n_trials_spec - 1: ax.set_xlabel('Time (s)', fontsize=9)
        else: ax.set_xticks([])

plt.suptitle('Trial-Specific Spectrograms', fontsize=12)
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'spectrogram_trial_specific.png'), dpi=150, bbox_inches='tight')
plt.show()

## Figure 5: Spectrogram Heatmaps - Trial-Averaged

In [ ]:
# ============================================================
# PLOT 5: Spectrogram Heatmaps - Trial-Averaged
# ============================================================

figure_style(font_size=8)

Z_gt_avg = np.abs(Z_gt).mean(axis=0)
Z_mt_avg = np.abs(Z_mt).mean(axis=0)
Z_lfp_avg = np.abs(Z_lfp).mean(axis=0) if HAS_LFP else None
Z_spk_avg = np.abs(Z_spk).mean(axis=0) if HAS_SPK else None

avg_data = [Z_gt_avg, Z_mt_avg]
avg_labels = ['Ground Truth', 'Multi-taper']
avg_scales = [1.0, scale_mt_global]
if HAS_LFP:
    avg_data.append(Z_lfp_avg); avg_labels.append('CT-SSMT (LFP)'); avg_scales.append(scale_lfp_global)
if HAS_SPK:
    avg_data.append(Z_spk_avg); avg_labels.append('CT-SSMT (Joint)'); avg_scales.append(scale_spk_global)
n_avg = len(avg_data)

fig, axes = plt.subplots(1, n_avg, figsize=(3.5*n_avg + 0.5, 3.5))
all_amps = [data[:, t_start_idx:t_end_idx] * scale for data, scale in zip(avg_data, avg_scales)]
vmin, vmax = 0, np.percentile(np.concatenate([a.flatten() for a in all_amps]), 99.5)

im = None
for j, (data, label, scale) in enumerate(zip(avg_data, avg_labels, avg_scales)):
    ax = axes[j]
    amp = data[:, t_start_idx:t_end_idx] * scale
    im = ax.imshow(amp, aspect='auto', origin='lower', extent=extent, cmap='viridis', vmin=vmin, vmax=vmax, interpolation='none')
    for freq_true in freqs_true:
        ax.axhline(freq_true, color='white', linestyle='--', lw=1.0, alpha=0.8)
    ax.set_title(label, fontsize=10, fontweight='bold')
    ax.set_xlabel('Time (s)', fontsize=9)
    if j == 0: ax.set_ylabel('Frequency (Hz)', fontsize=9)
    else: ax.set_yticks([])

fig.subplots_adjust(right=0.92)
cbar_ax = fig.add_axes([0.94, 0.15, 0.02, 0.7])
fig.colorbar(im, cax=cbar_ax, label='|Z| (scaled)')
plt.suptitle(f'Trial-Averaged Spectrograms (n={R} trials)', fontsize=12)
plt.savefig(os.path.join(OUTPUT_DIR, 'spectrogram_trial_averaged.png'), dpi=150, bbox_inches='tight')
plt.show()

## Figure 6: Deviation Spectrograms

In [ ]:
# ============================================================
# PLOT 6: Deviation Spectrograms
# ============================================================

figure_style(font_size=8)

Z_gt_dev_spec = np.abs(Z_gt) - Z_gt_avg[None, :, :]
Z_mt_dev_spec = np.abs(Z_mt) - Z_mt_avg[None, :, :]
Z_lfp_dev_spec = (np.abs(Z_lfp) - Z_lfp_avg[None, :, :]) if HAS_LFP else None
Z_spk_dev_spec = (np.abs(Z_spk) - Z_spk_avg[None, :, :]) if HAS_SPK else None

dev_data = [Z_gt_dev_spec, Z_mt_dev_spec]
dev_labels = ['GT Deviation', 'MT Deviation']
dev_scales = [1.0, scale_mt_global]
if HAS_LFP:
    dev_data.append(Z_lfp_dev_spec); dev_labels.append('LFP Deviation'); dev_scales.append(scale_lfp_global)
if HAS_SPK:
    dev_data.append(Z_spk_dev_spec); dev_labels.append('Joint Deviation'); dev_scales.append(scale_spk_global)
n_dev = len(dev_data)

fig, axes = plt.subplots(n_trials_spec, n_dev, figsize=(3.5*n_dev + 0.5, 2.5*n_trials_spec))

all_devs = [data[trial_idx, :, t_start_idx:t_end_idx] * scale 
            for data, scale in zip(dev_data, dev_scales) for trial_idx in SAMPLE_TRIALS_SPEC]
vmax = np.percentile(np.abs(np.concatenate([d.flatten() for d in all_devs])), 95)
vmin = -vmax

im = None
for i, trial_idx in enumerate(SAMPLE_TRIALS_SPEC):
    for j, (data, label, scale) in enumerate(zip(dev_data, dev_labels, dev_scales)):
        ax = axes[i, j]
        dev = data[trial_idx, :, t_start_idx:t_end_idx] * scale
        im = ax.imshow(dev, aspect='auto', origin='lower', extent=extent, cmap='RdBu_r', vmin=vmin, vmax=vmax, interpolation='none')
        for freq_true in freqs_true:
            ax.axhline(freq_true, color='black', linestyle='--', lw=0.8, alpha=0.7)
        if i == 0: ax.set_title(label, fontsize=10, fontweight='bold')
        if j == 0: ax.set_ylabel(f'Trial {trial_idx}\nFreq (Hz)', fontsize=9)
        else: ax.set_yticks([])
        if i == n_trials_spec - 1: ax.set_xlabel('Time (s)', fontsize=9)
        else: ax.set_xticks([])

fig.subplots_adjust(right=0.92)
cbar_ax = fig.add_axes([0.94, 0.15, 0.02, 0.7])
fig.colorbar(im, cax=cbar_ax, label='Deviation (scaled)')
plt.suptitle('Deviation Spectrograms: |Z_r| − mean(|Z|)', fontsize=12)
plt.savefig(os.path.join(OUTPUT_DIR, 'spectrogram_deviation.png'), dpi=150, bbox_inches='tight')
plt.show()

## Figure 7: Correlation Box Plot (COUPLED BANDS ONLY)

In [ ]:
# ============================================================
# PLOT 7: Correlation Distribution - COUPLED BANDS ONLY
# ============================================================

figure_style(font_size=8)

def compute_all_trial_correlations(Z_est, Z_gt, freq_indices, time_mask=None):
    """Compute correlation between estimate and ground truth for each trial."""
    R = Z_est.shape[0]
    n_freqs = len(freq_indices)
    correlations = np.zeros((R, n_freqs))
    for i, j_dense in enumerate(freq_indices):
        for r in range(R):
            if time_mask is not None:
                est = np.abs(Z_est[r, j_dense, time_mask])
                gt = np.abs(Z_gt[r, j_dense, time_mask])
            else:
                est = np.abs(Z_est[r, j_dense, :])
                gt = np.abs(Z_gt[r, j_dense, :])
            correlations[r, i] = np.corrcoef(est, gt)[0, 1]
    return correlations

t_mask = (time >= TIME_RANGE[0]) & (time <= TIME_RANGE[1])

# Only use COUPLED bands
corr_mt = compute_all_trial_correlations(Z_mt, Z_gt, idx_coupled_to_dense, t_mask)
corr_lfp = compute_all_trial_correlations(Z_lfp, Z_gt, idx_coupled_to_dense, t_mask) if HAS_LFP else None
corr_spk = compute_all_trial_correlations(Z_spk, Z_gt, idx_coupled_to_dense, t_mask) if HAS_SPK else None

import pandas as pd
data_list = []
for i, freq_hz in enumerate(freqs_coupled):
    for r in range(R):
        data_list.append({'Frequency (Hz)': f'{int(freq_hz)}', 'Correlation': corr_mt[r, i], 'Method': METHOD_CONFIG['mt']['label']})
        if HAS_LFP:
            data_list.append({'Frequency (Hz)': f'{int(freq_hz)}', 'Correlation': corr_lfp[r, i], 'Method': METHOD_CONFIG['lfp']['label']})
        if HAS_SPK:
            data_list.append({'Frequency (Hz)': f'{int(freq_hz)}', 'Correlation': corr_spk[r, i], 'Method': METHOD_CONFIG['spk']['label']})

df_corr = pd.DataFrame(data_list)

method_order = [METHOD_CONFIG['mt']['label']]
palette = {METHOD_CONFIG['mt']['label']: METHOD_CONFIG['mt']['color']}
if HAS_LFP:
    method_order.append(METHOD_CONFIG['lfp']['label'])
    palette[METHOD_CONFIG['lfp']['label']] = METHOD_CONFIG['lfp']['color']
if HAS_SPK:
    method_order.append(METHOD_CONFIG['spk']['label'])
    palette[METHOD_CONFIG['spk']['label']] = METHOD_CONFIG['spk']['color']

fig, ax = plt.subplots(figsize=(5, 3.5))
sns.boxplot(data=df_corr, x='Frequency (Hz)', y='Correlation', hue='Method',
            hue_order=method_order, palette=palette, width=0.6, linewidth=0.8, fliersize=2, ax=ax)

ax.set_xlabel('Frequency (Hz)', fontsize=9)
ax.set_ylabel('Correlation with Ground Truth', fontsize=9)
ax.set_ylim([-0.5, 1.05])
ax.axhline(0, color='gray', linestyle='--', linewidth=0.5, alpha=0.5)
ax.legend(title=None, loc='upper center', bbox_to_anchor=(0.5, -0.12), frameon=False, fontsize=7, ncol=3)
ax.set_title('Correlation with Ground Truth (Coupled Bands Only)', fontsize=10)

sns.despine()
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'correlation_boxplot.pdf'), dpi=150, bbox_inches='tight')
plt.savefig(os.path.join(OUTPUT_DIR, 'correlation_boxplot.png'), dpi=150, bbox_inches='tight')
plt.show()

print("\nCorrelation Summary Statistics (median ± IQR) - COUPLED BANDS:")
print("="*70)
for i, freq_hz in enumerate(freqs_coupled):
    print(f"\n{freq_hz:.0f} Hz:")
    print(f"  {METHOD_CONFIG['mt']['label']:<25}: {np.median(corr_mt[:, i]):.3f} ± {np.percentile(corr_mt[:, i], 75) - np.percentile(corr_mt[:, i], 25):.3f}")
    if HAS_LFP:
        print(f"  {METHOD_CONFIG['lfp']['label']:<25}: {np.median(corr_lfp[:, i]):.3f} ± {np.percentile(corr_lfp[:, i], 75) - np.percentile(corr_lfp[:, i], 25):.3f}")
    if HAS_SPK:
        print(f"  {METHOD_CONFIG['spk']['label']:<25}: {np.median(corr_spk[:, i]):.3f} ± {np.percentile(corr_spk[:, i], 75) - np.percentile(corr_spk[:, i], 25):.3f}")

## Figure 8: Power Spectral Density Comparison

In [ ]:
# ============================================================
# PLOT 8: Power Spectral Density Comparison
# Different line styles for coupled vs signal-only bands
# ============================================================

figure_style(font_size=8)

def compute_psd_stats(Z, axis_time=2):
    """Compute mean and std of power spectral density across trials."""
    power_per_trial = np.mean(np.abs(Z) ** 2, axis=axis_time)
    psd_mean = np.mean(power_per_trial, axis=0)
    psd_std = np.std(power_per_trial, axis=0)
    return psd_mean, psd_std

psd_mt_mean, psd_mt_std = compute_psd_stats(Z_mt)
psd_mt_mean *= (scale_mt_global ** 2)
psd_mt_std *= (scale_mt_global ** 2)

if HAS_LFP:
    psd_lfp_mean, psd_lfp_std = compute_psd_stats(Z_lfp)
    psd_lfp_mean *= (scale_lfp_global ** 2)
    psd_lfp_std *= (scale_lfp_global ** 2)

if HAS_SPK:
    psd_spk_mean, psd_spk_std = compute_psd_stats(Z_spk)
    psd_spk_mean *= (scale_spk_global ** 2)
    psd_spk_std *= (scale_spk_global ** 2)

fig, ax = plt.subplots(figsize=(6, 4))

ax.fill_between(FREQS_DENSE, psd_mt_mean - psd_mt_std, psd_mt_mean + psd_mt_std,
                color=METHOD_CONFIG['mt']['color'], alpha=0.2)
ax.plot(FREQS_DENSE, psd_mt_mean, color=METHOD_CONFIG['mt']['color'], 
        linewidth=3, label=METHOD_CONFIG['mt']['label'], zorder=2)

if HAS_LFP:
    ax.fill_between(FREQS_DENSE, psd_lfp_mean - psd_lfp_std, psd_lfp_mean + psd_lfp_std,
                    color=METHOD_CONFIG['lfp']['color'], alpha=0.2)
    ax.plot(FREQS_DENSE, psd_lfp_mean, color=METHOD_CONFIG['lfp']['color'], 
            linewidth=3, label=METHOD_CONFIG['lfp']['label'], zorder=3)

if HAS_SPK:
    ax.fill_between(FREQS_DENSE, psd_spk_mean - psd_spk_std, psd_spk_mean + psd_spk_std,
                    color=METHOD_CONFIG['spk']['color'], alpha=0.2)
    ax.plot(FREQS_DENSE, psd_spk_mean, color=METHOD_CONFIG['spk']['color'], 
            linewidth=3, label=METHOD_CONFIG['spk']['label'], zorder=4)

# Mark coupled bands with thick solid lines (green)
for freq_hz in freqs_coupled:
    ax.axvline(freq_hz, color='#2ECC71', linestyle='-', linewidth=2.5, alpha=0.8, zorder=1,
               label='Coupled to spikes' if freq_hz == freqs_coupled[0] else '')

# Mark signal-only bands with thick dashed lines (red)
if len(freqs_extra) > 0:
    for freq_hz in freqs_extra:
        ax.axvline(freq_hz, color='#E74C3C', linestyle='--', linewidth=2.5, alpha=0.8, zorder=1,
                   label='Signal-only (no coupling)' if freq_hz == freqs_extra[0] else '')

ax.set_xlabel('Frequency (Hz)', fontsize=10)
ax.set_ylabel('Mean Power (a.u.)', fontsize=10)
ax.set_xlim([FREQS_DENSE[0] - 1, FREQS_DENSE[-1] + 1])
ax.set_ylim(bottom=0)
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), frameon=False, fontsize=8, ncol=3)

sns.despine()
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'psd_comparison.pdf'), dpi=150, bbox_inches='tight')
plt.savefig(os.path.join(OUTPUT_DIR, 'psd_comparison.png'), dpi=150, bbox_inches='tight')
plt.show()

# SNR Analysis
def compute_snr_metrics(psd, freq_indices, all_freqs):
    signal_mask = np.zeros(len(all_freqs), dtype=bool)
    signal_mask[freq_indices] = True
    power_signal = np.mean(psd[signal_mask])
    power_noise = np.mean(psd[~signal_mask])
    snr = power_signal / power_noise if power_noise > 0 else np.inf
    return power_signal, power_noise, snr

print("\nSNR Analysis (Signal vs. Off-Signal Frequencies):")
print("="*70)
print(f"{'Method':<30} {'P_signal':>12} {'P_noise':>12} {'SNR':>10}")
print("-"*70)

p_sig_mt, p_noise_mt, snr_mt = compute_snr_metrics(psd_mt_mean, idx_true_to_dense, FREQS_DENSE)
print(f"{METHOD_CONFIG['mt']['label']:<30} {p_sig_mt:>12.4f} {p_noise_mt:>12.6f} {snr_mt:>10.1f}")

if HAS_LFP:
    p_sig_lfp, p_noise_lfp, snr_lfp = compute_snr_metrics(psd_lfp_mean, idx_true_to_dense, FREQS_DENSE)
    print(f"{METHOD_CONFIG['lfp']['label']:<30} {p_sig_lfp:>12.4f} {p_noise_lfp:>12.6f} {snr_lfp:>10.1f}")

if HAS_SPK:
    p_sig_spk, p_noise_spk, snr_spk = compute_snr_metrics(psd_spk_mean, idx_true_to_dense, FREQS_DENSE)
    print(f"{METHOD_CONFIG['spk']['label']:<30} {p_sig_spk:>12.4f} {p_noise_spk:>12.6f} {snr_spk:>10.1f}")

print("-"*70)
print("\nSNR Improvement over Multi-taper:")
if HAS_LFP:
    print(f"  {METHOD_CONFIG['lfp']['label']}: {snr_lfp / snr_mt:.2f}x")
if HAS_SPK:
    print(f"  {METHOD_CONFIG['spk']['label']}: {snr_spk / snr_mt:.2f}x")